In [ ]:
import numpy as np
import random as r
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend

In [ ]:
# const
BITS = 7
MASK = (1<<BITS) - 1
MIN_FATIGUE=1


def ones(value):
    n = 0
    while value:
        n += 1
        value &= value-1
    return n

def threshold(value, window, adapt):

    mask = (1<<window)-1
    value = ones(value & mask)
    return value >= adapt

class Neuron:
    def __init__(self, min_threshold, uid):
        self.uid = uid
        self.state = 0
        self.min_threshold = min_threshold
        self.threshold = min_threshold
        self.recovery = 0
        self.targets = []
        self.inbox = 0
        
        self.fired = 0
    
    def enqueue(self):
        self.inbox = 1

    def process(self, epoch):
        signal = self.inbox
        self.inbox = 0
        
        def log(msg):
            if self.uid == 0:
                print(msg)
                
        
        self.state = ((self.state << 1) | signal) & MASK


        self.threshold = min(self.threshold, BITS)
        activate = ones(self.state & MASK) >= self.threshold

        if activate:
            for t in self.targets:
                t.enqueue()
            #O[k] = 1
            self.state = 0
            self.threshold += 1
            self.recovery=0
            self.fired = 1

        else:
            self.fired = 0

        if self.recovery>=8:
            self.recovery=0
            self.threshold=max(self.min_threshold, self.threshold-1)    

        if self.threshold:
            self.recovery+=1
        
    pass


class Cleft:
    def __init__(self, neuron):
        self.neuron = neuron
        
        self.inbox = 0
        
    def enqueue(self):
        self.inbox = 1
        
    def process(self, epoch):
        if self.inbox:
            self.neuron.enqueue()
            self.inbox = 0

In [ ]:
neurons = [Neuron((i % 2)+1, i) for i in range(1000)]
# connect neurons randomly

clefts = []
for n in neurons:
    targets = r.sample(neurons, 3)
    for t in targets:
        c = Cleft(t)
        n.targets.append(c) 
        clefts.append(c)

fired = 0

for n in neurons[0:10]:
    n.enqueue()
for i in range(100):
    neuron_inbox = 0
    cleft_inbox = 0
    for n in neurons:
        if n.inbox:
            neuron_inbox+=1
        n.process(i)
        if n.fired:
            fired += 1
    for c in clefts:
        if c.inbox:
            cleft_inbox +=1
        c.process(i)
    
        
    print(f"N:{neuron_inbox} C:{cleft_inbox}")
        